<a href="https://colab.research.google.com/github/antontmur/graph_search_algorithms/blob/master/Graph%20Search%20Algorithms%20(rus).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт

In [25]:
from google.colab import drive
drive.mount('/content/gdrive/')
home_dir = '/content/gdrive/My Drive/Colab Notebooks/praktikum/'
import os
os.chdir(home_dir)

from math import sqrt
from utils import generate_graph, generate_maze, print_path
from graph_animation import GraphAnimation

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


# Ищем путь в небольшом графе

## Создаём задачу

In [ ]:
# Создаём простой граф, в котором будем искать путь от одного узла к другому
graph = generate_graph()

# Задаём начальный и конечный узлы графа
start_node = 0
goal_node = 10

# Наш помощник для создания анимаций
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=False, maze_list=[])
fig = graph_animation.make_one_shot()
fig.show()

## Вспомогательные классы. 
Структуры данных под разные алгоритмы.

### Стек и очередь.

In [27]:
class Stack:
    '''Обычный стек, устроенный по принципу LIFO - Last In First Out'''

    def __init__(self):
        self.data = []

    def insert(self, elem):
        self.data.append(elem)

    def get_first(self):
        return self.data.pop()

    def empty(self):
        return len(self.data) == 0


class Queue:
    '''Обычная очередь, устроенная по принципу FIFO - First In First Out'''

    def __init__(self):
        self.data = []

    def insert(self, elem):
        self.data.append(elem)

    def get_first(self):
        return self.data.pop(0)

    def empty(self):
        return len(self.data) == 0


### Очередь с приоритетом для алгоритма Дейкстры

In [28]:
class DijkstraQueue:
    '''Очередь с приоритетом для метода Дейкстры. 
       В методе get_first() выбирается узел, 
       до которого расстояние от начального узла минимально'''

    def __init__(self, g, dist):
        self.data = []
        self.graph = g
        self.dist = dist

    def insert(self, elem):
        self.data.append(elem)

    def get_first(self):
        current_minimum = float('Inf')
        current_minimum_vertex = None
        for node in self.data:
            if self.dist[node] < current_minimum:
                current_minimum = self.dist[node]
                current_minimum_vertex = node
        self.data.remove(current_minimum_vertex)
        return current_minimum_vertex

    def empty(self):
        return len(self.data) == 0

### Очередь с приоритетом для алгоритма A*

In [29]:
class AStarQueue:
    '''Очередь с приоритетом для метода Дейкстры. 
       В методе get_first() выбирается узел, 
       для которого минимальна сумма:
       расстояние от начального узла + оценка расстояния до конечного узла'''

    def __init__(self, g, dist, goal_node):
        self.data = []
        self.graph = g
        self.x_goal, self.y_goal = graph.nodes[goal_node]['pos']
        self.goal_node = goal_node
        self.dist = dist

    def insert(self, elem):
        self.data.append(elem)

    def calc_euristic(self, node):
        x_node, y_node = graph.nodes[node]['pos']
        dist_to_goal = sqrt((x_node-self.x_goal)**2 + (y_node-self.y_goal)**2)
        return self.dist[node] + dist_to_goal

    def get_first(self):
        current_minimum = float('Inf')
        current_minimum_vertex = None
        for node in self.data:
            euristic = self.calc_euristic(node)
            if euristic < current_minimum:
                current_minimum = euristic
                current_minimum_vertex = node
        self.data.remove(current_minimum_vertex)
        return current_minimum_vertex

    def empty(self):
        return len(self.data) == 0


## Алгоритм обхода графа в поисках целевого узла

In [30]:
def find_path(graph, start_node, goal_node, data_structure='Stack'):

    # Цвета узлов, расстояние от начала
    color = ['white'] * graph.number_of_nodes()
    dist = [float('Inf')] * graph.number_of_nodes()
    parent = dict()

    # Узлы, которые мы хотим посетить
    # Выбор структуры данных, которую мы будем использовать для Q 
    # зависит от входной переменной data_structure
    Q = {'Stack' : Stack(),
         'Queue' : Queue(),
         'DijkstraQueue' : DijkstraQueue(graph, dist),
         'AStarQueue' :AStarQueue(graph, dist, goal_node)}[data_structure]

    # Начинаем со стартового узла
    Q.insert(start_node)
    color[start_node] = 'black'
    graph_animation.add_frame(color, parent, start_node)
    dist[start_node] = 0

    # Цикл, пока в Q не закончатся узлы
    while not Q.empty():
        current_node = Q.get_first()

        if current_node == goal_node:
            print('SUCCESS !')
            print_path(goal_node, parent)
            graph_animation.add_frame(color, parent, current_node)
            break

        # Берём соседей текущего узла
        neighbours = list(graph.adj[current_node])
        for node_to_go in neighbours:
            if color[node_to_go] is 'white':
                # Если это новый узел
                color[node_to_go] = 'grey'                 # Красим в серый
                Q.insert(node_to_go)                       # Добавляем его в "очередь"
                parent[node_to_go] = current_node          # Запоминаем родителя
                dist[node_to_go] = dist[current_node] + graph.get_edge_data(node_to_go, current_node)['weight']
            else:
                # Иначе решаем конфликт
                resolve_duplicate(node_to_go, current_node, dist)

        color[current_node] = 'black'                      # Закончили с этим узлом - красим его в чёрный
        graph_animation.add_frame(color, parent, current_node)


def resolve_duplicate(node_to_go, current_node, dist):
    weight = graph.get_edge_data(node_to_go, current_node)['weight']
    if dist[current_node] + weight < dist[node_to_go]:
        dist[node_to_go] = dist[current_node] + weight
    return 0

## Depth-First Search

In [ ]:
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=False, maze_list=[])

# Если мы хотим обходить граф, используя DFS, выбираем структуру данных - Стек
find_path(graph, start_node, goal_node, data_structure='Stack')
fig = graph_animation.make_animation()
fig.show()

## Breadth-First Search

In [ ]:
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=False, maze_list=[])

# Если мы хотим обходить граф, используя BFS, 
# выбираем структуру данных - Очередь
find_path(graph, start_node, goal_node, data_structure='Queue')
fig = graph_animation.make_animation()
fig.show()

## Алгоритм Дейкстры

In [ ]:
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=False, maze_list=[], edge_weight=True)

# Если мы хотим обходить граф, используя алгоритм Дейкстры, 
# то используем соответствующую структуру данных - очередь с приоритетом
find_path(graph, start_node, goal_node, data_structure='DijkstraQueue')
fig = graph_animation.make_animation()
fig.show()

# Поиск пути в лабиринте

In [ ]:
# Создём новую задачу.
# На этот раз большой лабиринт
graph, maze_list = generate_maze()
start_node = 113
goal_node = 198

graph_animation = GraphAnimation(graph, start_node, goal_node, 
                                 is_maze=True, maze_list=maze_list)
fig = graph_animation.make_one_shot()
fig.show()

## Проверим, как работает алгоритм Дейкстры

In [ ]:
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=True, maze_list=maze_list)

# Если мы хотим обходить граф, используя алгоритм Дейкстры, 
# то используем соответствующую структуру данных - очередь с приоритетом
find_path(graph, start_node, goal_node, data_structure='DijkstraQueue')
fig = graph_animation.make_animation()
fig.show()

## И сравним с алгоритмом А*

In [ ]:
graph_animation = GraphAnimation(graph, start_node, goal_node,
                                 is_maze=True, maze_list=maze_list)

# Для поиска с помощь алгоритма А* будем использовать очередь,
# в которой приоритет определяется с учётом оценки расстояния до goal
find_path(graph, start_node, goal_node, data_structure='AStarQueue')
fig = graph_animation.make_animation()
fig.show()